<a href="https://colab.research.google.com/github/DavidScanu/oc-ai-engineer-p03-preparez-des-donnees-pour-un-organisme-de-sante-publique/blob/main/p03_scanu_david_01_notebook_2024_09_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/DavidScanu/oc-projet-02-smartcity/blob/main/images/logo-oc-github-banner.png?raw=true" />

# Projet 03 - Préparez des données pour un organisme de santé publique

🎓 OpenClassrooms - Parcours : [AI Engineer](https://openclassrooms.com/fr/paths/795-ai-engineer) | 👋 *Etudiant* : [David Scanu](https://www.linkedin.com/in/davidscanu14/)

## 📝 Contexte

- L'agence Santé publique France souhaite **améliorer sa base de données Open Food Facts** et fait appel aux services de votre entreprise.
- L’agence Santé publique France confie à votre entreprise **la création d’un système de suggestion ou d’auto-complétion** pour aider les usagers à remplir plus efficacement la base de données.

## Mission : nettoyer et explorer les données

1. **Traiter le jeu de données**
  - Repérer des variables pertinentes
  - Nettoyer les données
    - Valeurs manquantes
    - Valeurs aberrantes
  - Automatiser ces traitements
2. **Produire des visualisations**
  - Effectuer une analyse univariée pour chaque variable intéressante
3. **Analyse multivariée**
  - Sélectionner / créer des variables
  - Effectuer les tests statistiques appropriés
4. **Rédiger un rapport** d’exploration et une conclusion
5. Expliquer dans une présentation en quoi ce projet respecte les **5 grands principes du RGPD**.


## 💾 Jeux de données

<a href="https://fr.openfoodfacts.org/" target="_blank"><img src="https://static.openfoodfacts.org/images/logos/off-logo-horizontal-light.svg" width=260 /></a>

- https://world.openfoodfacts.org/data
- [Description du jeu de données Open Food Facts](https://world.openfoodfacts.org/data/data-fields.txt)
- https://wiki.openfoodfacts.org/Reusing_Open_Food_Facts_Data#The_CSV_daily_export

Les champs sont séparés en quatre sections :

1. **Informations générales** sur la fiche du produit : nom, date de modification, etc.
2. **Tags** : catégorie du produit, localisation, origine, etc.
3. **Ingrédients** composant les produits et leurs additifs éventuels
4. **Informations nutritionnelles** : quantité en grammes d’un nutriment pour 100 grammes du produit


## 🎯 Objectifs pédagogiques

- Déterminer les objectifs du nettoyage des données et de la démarche de mise en œuvre
- Effectuer des analyses statistiques univariées et multivariées
- Effectuer des opérations de nettoyage sur des données structurées
- Représenter des données grâce à des graphiques afin justifier les analyses réalisées


## 📦 Livrables

- [Présentation Google Slide]()
- [Notebook Colab](https://colab.research.google.com/drive/10W-7Lg2_5gn00mt5xfKLOc80I4wpa1_-?usp=sharing)
- [Dépôt GitHub]()

## Inspiration

- https://github.com/fleuryc/oc_ingenieur-ia_P3-Preparez-des-donnees-pour-un-organisme-de-sante-publique/blob/main/notebook.ipynb
- https://www.kaggle.com/code/hamadizarrouk/fork-of-hamadi-zarrouk-p3-01-notebook
- https://github.com/eleplanois/openclassRoom/blob/main/Projet_3%20Concevez%20une%20application%20au%20service%20de%20la%20sant%C3%A9%20publique/PSant%C3%A9_01_notebooknettoyage.ipynb

## ⚙️ Installation des bibliothèques nécessaires

In [ ]:
# !pip install plotly==5.24.0 --no-cache-dir --quiet

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 🥗 Importation du jeu de données

### Téléchargement et extraction du fichier .zip

Les données mises à disposition sont issues de [Open Food Facts](https://world.openfoodfacts.org/) et présentent les données sur les produits alimentaires.

Nous téléchargeons et extrayons le fichier ZIP.

In [ ]:
!wget "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip" -O temp.zip
!unzip temp.zip -d /content/data
!rm temp.zip

--2024-09-22 09:01:04--  https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.29.147, 52.92.33.8, 52.218.37.211, ...
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.29.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88117814 (84M) [application/x-www-form-urlencoded]
Saving to: ‘temp.zip’

temp.zip            100%[===================>]  84.04M  23.5MB/s    in 3.6s    

2024-09-22 09:01:08 (23.5 MB/s) - ‘temp.zip’ saved [88117814/88117814]

Archive:  temp.zip
  inflating: /content/data/fr.openfoodfacts.org.products.csv  
   creating: /content/data/__MACOSX/
  inflating: /content/data/__MACOSX/._fr.openfoodfacts.org.products.csv  


### Correction d'irrégularités dans le fichier CSV

Nous allons corriger des irrégularités en supprimant les sauts de ligne superflus, puis écrire les données propres dans un nouveau fichier CSV.

In [ ]:
data_folderpath = "/content/data/"
raw_csv_filepath = data_folderpath + "fr.openfoodfacts.org.products.csv"
clean_csv_filepath = data_folderpath + "fr.openfoodfacts.org.products-clean.csv"

In [ ]:
import os

if not os.path.isfile(clean_csv_filepath):
    # Seulement si le fichier clean_csv n'est pas déjà présent
    with open(raw_csv_filepath, 'r') as csv_file, open(clean_csv_filepath, 'w') as clean_file:
        """
            Gérer les irrégularités

            23 points de données sont divisés à tort en deux lignes :
            - lignes : 189070, 189105, 189111, 189121, 189154, 189162, 189164, 189170, 189244, 189246,
                    189250, 189252, 189262, 189264, 189271, 189274, 189347, 189364, 189366, 189381,
                    189406, 189408, 189419

            Le schéma est toujours le même :
            - un caractère NewLine (`\n`) est placé à la fin de la colonne "first_packaging_code_geo"
            - et la ligne suivante commence par un séparateur TAB (`\t`) : la colonne "villes" est vide.

            Puisque la première colonne (« code ») n'est jamais vide, nous supprimons simplement tout caractère `\n` qui est
            directement suivi d'un séparateur TAB (`\t`).
        """
        data = csv_file.read()
        clean_file.write(data.replace('\n\t', '\t'))

### Chargement du ficher .csv dans un DataFrame

- Chargement des données dans un DataFrame
- Conversion des variables au bon **dtype**, avec l'aide de la [description des données fournie](https://static.openfoodfacts.org/data/data-fields.txt).

In [ ]:
# Liste des noms des colonnes
column_names = pd.read_csv(clean_csv_filepath, sep='\t', encoding='utf-8', nrows=0, low_memory=False, on_bad_lines='error').columns.values

# Définir les types de colonnes en fonction de la description des champs (https://static.openfoodfacts.org/data/data-fields.txt)
column_types = {col: 'Int64' for (col) in column_names if col.endswith(('_t', '_n'))}
column_types |= {col: float for (col) in column_names if col.endswith(('_100g', '_serving'))}
column_types |= {col: str for (col) in column_names if not col.endswith(('_t', '_n', '_100g', '_serving', '_tags'))}

tags_converter = lambda list_as_string_value : list_as_string_value.split(',') if list_as_string_value else pd.NA

# Charger les données brutes
raw_data = pd.read_csv(
  clean_csv_filepath,
  sep='\t',
  encoding='utf-8',
  low_memory=False,
  on_bad_lines='error',
  dtype=column_types,
  parse_dates=[col for (col) in column_names if col.endswith('_datetime')],
  converters={
      # Convertir les colonnes '_tags' en liste de valeurs (séparateur : ',')
      col: tags_converter
      for (col) in column_names if col.endswith('_tags')
  }
)

# Afficher la taille du DataFrame
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320749 entries, 0 to 320748
Columns: 162 entries, code to water-hardness_100g
dtypes: Int64(5), datetime64[ns, UTC](2), float64(99), object(56)
memory usage: 398.0+ MB


## 👀 Visualisation générale du jeu de données

In [ ]:
# Vue globale
def nb_lines(data):
    ''' returns number of rows'''
    return len(data)

def nb_columns(data):
    ''' returns number of columns'''
    return len(data.columns)

def missing_cells(data):
    ''' returns number of missing cells'''
    return data.isna().sum().sum()

def missing_cells_percent(data):
    ''' returns percentage of missing cells'''
    return data.isna().sum().sum()/(data.size)

def data_set_overview(data):
    '''  prints a dataframe summary containing:number of rows, columns, missing cells and duplicate
    rows'''

    print('***********************************')
    print('Data : {}'.format(namestr(data, globals())))
    print('Nombre de variables : {}'.format(nb_columns(data)))
    print('Nombre de lignes : {}'.format(nb_lines(data)))
    print('Valeurs manquantes : {}'.format(missing_cells(data)))
    print('Valeurs manquantes en % : {:.2%}'.format(missing_cells_percent(data)))
    print('***********************************')
    return None

data_set_overview(raw_data)

In [ ]:
raw_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320749 entries, 0 to 320748
Data columns (total 162 columns):
 #    Column                                      Dtype              
---   ------                                      -----              
 0    code                                        object             
 1    url                                         object             
 2    creator                                     object             
 3    created_t                                   Int64              
 4    created_datetime                            datetime64[ns, UTC]
 5    last_modified_t                             Int64              
 6    last_modified_datetime                      datetime64[ns, UTC]
 7    product_name                                object             
 8    generic_name                                object             
 9    quantity                                    object             
 10   packaging                                 

In [ ]:
raw_data.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_fr,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_fr,main_category,main_category_fr,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,omega-6-fat_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17 09:17:46+00:00,1474103893,2016-09-17 09:18:13+00:00,Farine de blé noir,NaN,1kg,NaN,<NA>,Ferme t'y R'nao,[ferme-t-y-r-nao],NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,en:FR,[en:france],France,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","[en:to-be-completed, en:nutrition-facts-to-be-...","A compléter,Informations nutritionnelles à com...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09 14:32:37+00:00,1489069957,2017-03-09 14:32:37+00:00,Banana Chips Sweetened (Whole),NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,US,[en:united-states],États-Unis,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,NaN,NaN,<NA>,N

## Sélection des variables pertinentes

Nous allons chercher à n'utiliser que les variables pertinentes pour SPF :
- Celles pour lesquelles nous avons **suffisament de valeurs non vides** pour pouvoir faire une analyse statistique fiable.
- Celles qui peuvent avoir un réel sens du point de vue des problématiques de santé publique : **variables utilisées pour le calcul du [Nutri Score](https://www.santepubliquefrance.fr/determinants-de-sante/nutrition-et-activite-physique/articles/nutri-score)**.

> Filtrer par pays

In [ ]:
def missing_values(data):
    """
    Function that takes dataframe as input and output a dataframe containing variables, number of missing values and % of missing values.
    - param
        data: dataframe
    - return:
        dataframe
    """
    summary = pd.DataFrame(columns=['Nom de la variable', 'Nb valeurs manquantes', '% valeurs manquantes'])
    summary['Nom de la variable']=data.columns
    missing = list()
    percent_missing = list()
    for var in data.columns:
        nb_missing = missing_cells(data[var])
        pc_missing = missing_cells_percent(data[var])
        missing.append(nb_missing)
        percent_missing.append(pc_missing)
    summary['Nb valeurs manquantes'] = list(missing)
    summary['% valeurs manquantes'] = list(percent_missing)
    summary.sort_values(by=['% valeurs manquantes'], ascending=True, inplace=True)
    # summary.reset_index(drop=True, inplace=True)
    return summary

In [ ]:
# pourcentage des données manquantes
missing_values(raw_data)

,Nom de la variable,Nb valeurs manquantes,% valeurs manquantes
0,code,0,0.00
1,url,0,0.00
2,creator,2,0.00
3,created_t,0,0.00
4,created_datetime,1,0.00
5,last_modified_t,0,0.00
6,last_modified_datetime,0,0.00
58,states_fr,0,0.00
57,states_tags,0,0.00
56,states,0,0.00


In [ ]:
# Let's define a function to reuse this graph later
def plot_empty_values(dataframe: pd.DataFrame) -> None:
    """
    Tracer un histogramme du pourcentage de valeurs vides par colonnes du DataFrame d'entrée
    """
    num_rows = len(dataframe.index)
    columns_emptiness = pd.DataFrame({
        col : {
            'count': dataframe[col].isna().sum(),
            'percent': 100 * dataframe[col].isna().sum() / num_rows,
        } for col in dataframe.columns
    }).transpose().sort_values(by=['count'])

    fig = px.bar(columns_emptiness,
        color='percent',
        y='percent',
        labels={
            'index':'column name',
            'percent':'% de valeurs vides',
            'count':'# de valeurs vides',
        },
        hover_data=['count'],
        title='Valeurs vides par colonne',
        width=1200,
        height=600,
    )
    fig.show()

plot_empty_values(raw_data)

In [ ]:
# Let's keep only meaningful columns
meaningful_columns = [
    # General information
    'code', 'product_name', 'main_category', 'additives_n',

    # Nutri-Score
    'nutrition_grade_fr', 'nutrition-score-fr_100g',

    # Positive nutrition facts
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g',

    # Negative nutrition facts
    'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g',
]
meaningful_data = raw_data.loc[:, meaningful_columns].copy()

# Display DataFrame size
meaningful_data.info()

# Display first values of each column
meaningful_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320749 entries, 0 to 320748
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   code                         320749 non-null  object 
 1   product_name                 302987 non-null  object 
 2   main_category                84389 non-null   object 
 3   additives_n                  248961 non-null  Int64  
 4   nutrition_grade_fr           221233 non-null  object 
 5   nutrition-score-fr_100g      221233 non-null  float64
 6   energy_100g                  261136 non-null  float64
 7   saturated-fat_100g           229577 non-null  float64
 8   sugars_100g                  244994 non-null  float64
 9   salt_100g                    255533 non-null  float64
 10  fruits-vegetables-nuts_100g  3046 non-null    float64
 11  fiber_100g                   200891 non-null  float64
 12  proteins_100g                259929 non-null  float64
dtyp

,code,product_name,main_category,additives_n,nutrition_grade_fr,nutrition-score-fr_100g,energy_100g,saturated-fat_100g,sugars_100g,salt_100g,fruits-vegetables-nuts_100g,fiber_100g,proteins_100g
0,0000000003087,Farine de blé noir,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,Banana Chips Sweetened (Whole),NaN,0,d,14.0,2243.0,28.57,14.29,0.00000,NaN,3.6,3.57
2,0000000004559,Peanuts,NaN,0,b,0.0,1941.0,0.00,17.86,0.63500,NaN,7.1,17.86
3,0000000016087,Organic Salted Nut Mix,NaN,0,d,12.0,2540.0,5.36,3.57,1.22428,NaN,7.1,17.86
4,0000000016094,Organic Polenta,NaN,0,NaN,NaN,1552.0,NaN,NaN,NaN,NaN,5.7,8.57


In [ ]:
# pourcentage des données manquantes
missing_values(meaningful_data)

,Nom de la variable,Nb valeurs manquantes,% valeurs manquantes
0,code,0,0.00
1,product_name,17762,5.54
2,main_category,236360,73.69
3,additives_n,71788,22.38
4,nutrition_grade_fr,99516,31.03
5,nutrition-score-fr_100g,99516,31.03
6,energy_100g,59613,18.59
7,saturated-fat_100g,91172,28.42
8,sugars_100g,75755,23.62
9,salt_100g,65216,20.33
